In [1]:
import numpy as np
import sklearn
import pandas as pd
from tritraining import TriTraining
from tritraining import TriTrainingwDisagreement
from selftraining import SelfTraining
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
classifiers = [[]]*3

from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn import tree

from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB,BernoulliNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier


classifiers[0] = RandomForestClassifier(class_weight='balanced')
classifiers[1] = SVC()
classifiers[2] = SGDClassifier()

In [3]:
tweets_label_train_df = pd.read_csv("clean_tweets_labelled_train.csv")
tweets_label_test_df = pd.read_csv("clean_tweets_labelled_test.csv")
tweets_unlabel_df = pd.read_csv("tweets_unlabelled_cleaned_1L.csv")
#weets_unlabel_df=tweets_unlabel_df.iloc[:len(tweets_label_train_df), :]

In [4]:
tweets_label_train_df

,label,clean_data
0,0,day die call covid vaccin report
1,1,pranam messag today manifest dhyan
2,1,hyderabad base sought fund govern ramp product...
3,0,confirm chines high protect rate accord direct...
4,2,lab studi suggest vaccin protect
...,...,...
4195,1,seek full fda approv month account us dose giv...
4196,2,across countri peopl sign covid vaccin hope br...
4197,1,astrazeneca vaccin countri suspend ban open do...
4198,1,sindh high court allow thursday sale russia sp...


In [5]:
def data_process():
    #from sklearn.model_selection import train_test_split
    #X#_train, X_test, y_train, y_test = train_test_split(data, label, test_size = test_rate, random_state=0)

    
    rng = np.random.RandomState(42)
   # labeled_index = rng.rand(len(y_train)) < label_rate
    #unlabeled_index = np.logical_not(labeled_index)
   
    tfidf_vect = TfidfVectorizer( max_features=10000,
        sublinear_tf=True,
        strip_accents='unicode',
        stop_words='english',
        lowercase=True,
        analyzer='word',
        token_pattern=r'\w{3,}',
        ngram_range=(1, 3),
        dtype=np.float32,
        norm='l2',
        min_df=3,
        max_df=.9)
    tfidf_vect.fit_transform(tweets_unlabel_df['clean_data'].values.astype('U'))
    
    train_label_tfidf_vect = tfidf_vect.transform(tweets_label_train_df['clean_data'].values.astype('U'))

    L_X = train_label_tfidf_vect.todense()#data of L
    print(L_X.shape)
    L_y = tweets_label_train_df['label']#lable of L
    
    #tfidf_vect1 = TfidfVectorizer()
    train_unlabel_tfidf_vect = tfidf_vect.transform(tweets_unlabel_df['clean_data'].values.astype('U'))
    #print(train_unlabel_tfidf_vect)

    print(train_unlabel_tfidf_vect.shape)
    U_X = train_unlabel_tfidf_vect.todense()   #data of U
    print(U_X.shape)
   
    #tfidf_vect2 = TfidfVectorizer()
    #tfidf_vect2.fit(tweets_label_test_df['clean_data'].values.tolist())
    test_unlabel_tfidf_vect = tfidf_vect.transform(tweets_label_test_df['clean_data'].values.astype('U'))
    
    
    X_test = test_unlabel_tfidf_vect.todense()#data of L

    y_test = tweets_label_test_df['label']#lable of L
    return L_X, L_y, U_X, X_test, y_test

In [7]:
 L_X, L_y, U_X, X_test, y_test = data_process()
m = TriTraining(classifiers)

(4200, 10000)
(94947, 10000)
(94947, 10000)


In [8]:
m.classifiers

[RandomForestClassifier(class_weight='balanced'), SVC(), SGDClassifier()]

In [9]:
m.fit(L_X, L_y, U_X)

i is %s 0
hello Adding
i is %s 1
hello Adding
i is %s 2
hello Adding
i is %s 1
hello Adding
i is %s 2
hello Adding


In [10]:
proxy_labels=m.predict(U_X)

In [11]:
#m.score(X_test, y_test)

In [12]:
np.unique(proxy_labels, axis=0)

array([0, 1, 2])

In [13]:
len(proxy_labels)

94947

In [16]:
#add 'blocks' array as new column in DataFrame
tweets_unlabel_df['label'] = proxy_labels.tolist()

tweets_unlabel_df

,clean_data,label
0,folks said daikon paste could treat cytokine s...,1
1,world wrong side history year hopefully bigges...,1
2,russian vaccine created last 24 years,1
3,explain need vaccine,1
4,anyone useful adviceguidance whether covid vac...,1
...,...,...
94942,tema batch arrives,1
94943,plans allow vaccinated people wear maskfor lon...,1
94944,get vaccinated protect amp others,2
94945,get vaccinated today,1


In [17]:
tweets_unlabel_df.to_csv('tweets_unlabelled_1L_proxy_labels.csv', encoding='utf-8', index=False)

In [18]:
m.score(X_test, y_test)

0.67

In [19]:
! ps -ef | grep python

jovyan        15       7  0 Nov01 ?        00:00:33 /opt/conda/bin/python /opt/conda/bin/jupyterhub-singleuser --allow-root --ip=0.0.0.0 --port=8888 --SingleUserNotebookApp.default_url=/lab
jovyan       443      15 80 05:38 ?        04:47:18 /opt/conda/bin/python -m ipykernel_launcher -f /home/jovyan/.local/share/jupyter/runtime/kernel-b06116d4-43e6-4b68-81b0-61681c238dfb.json
jovyan       922     443  0 11:36 pts/1    00:00:00 /bin/bash -c  ps -ef | grep python
jovyan       924     922  0 11:36 pts/1    00:00:00 grep python
